# Un peu de théorie sur k-PPV

## Motivation intuitive

L'algorithme des $k$ plus proches voisins ($k$-PPV) est certainement un des algorithmes les plus simples d'apprentissage automatique. Il est motivé par l'idée que des *entrées* $x_t$ semblables devraient avoir des *cibles* $y_t$ semblables. Ainsi, pour bien définir un algorithme $k$-PPV, il suffit de définir ce que veut dire *semblable* dans le contexte des entrées et de définir l'influence de ces voisins sur la prédiction de la cible pour une entrée de test.

Donc, pour obtenir une prédiction de la cible pour une entrée de test $x$, il suffit de trouver les k plus proches voisins selon une métrique déterminant jusqu'à quel point des entrées sont semblables (par exemple, la distance euclédienne ou norme $L^2$, ou de façon plus générale la norme $L^p$ de Minkowski) et d'utiliser ces $k$ plus proches voisins pour prédire la cible de $x$. Dans un problème de classification, la prédiction correspond à la classe majoritaire parmi les $k$ plus proches voisins, i.e. que l'ensemble des $k$ plus proches voisins votent pour la classe correspondant à leur cible respective et la classe recueillant le plus de vote est choisie en tant que prédiction par l'algorithme.

## Formalisation mathématique

Soit

* $x$ une entrée de test
* $m$ le nombre de classes
* $D_n = \{(x_t,y_t)\}_{t=1}^n$ l'ensemble d'entraînement, où $y \in Y=\{1,\dots,m\}$ correspond à l'identité de la classe cible de l'entrée $x_t$
* $d(\dot{},\dot{})$ une fonction de distance
* $V(x,T,d(\dot{},\dot{}),k)$ be the $k$ nearest neighbours to $x$ from the examples $T$ (includes the example and the corresponding target class)

La prédiction de classification par l'algorithme des k plus proches voisins est donc:

> $$f(x)={\mbox{arg max}} \left(\frac{1}{k} \sum_{(x_i,y_i) \in V(x)} \mathrm{onehot}_{m}(y_i)\right)$$

Une fonction de distance couramment utilisée est la distance euclédienne:

> $$d(a,b)= \sqrt{\sum_{i=1}^d(a_i-b_i)^2}$$

qui est un cas spécifique, avec $p=2$, de la norme $L^p$ de Minkowski:

> $$d(a,b)= \left(\sum_{i=1}^d|a_i-b_i|^p\right)^\frac{1}{p}$$

## Pseudo-code

On définit un algorithme d'apprentissage en précisant sa procédure d'entraînement et de prédiction pour une nouvelle entrée de test. Étant donné que la procédure d'entraînement de l'algorithme $k$-PPV consiste simplement à mettre en mémoire l'ensemble d'entraînement $D_n$, voici donc la procédure de prédiction dans le cas particulier ou k=1:

    definition 1-PPV(x)
        min = +inf # Initialisation de la distance du plus proche voisin
        ppv = -1 # Initialisation de l'indice du plus proche voisin
        
        pour t=1 à n faire
            dt = d(X[t], x)
            si dt < min alors
                min = dt
                ppv = t
                
        retourner Y[ppv]
        
La prédiction s'exécute en temps $O(n(k+d))$. Il est cependant possible d'obtenir un temps d'exécution dans $O(n(log(k)+d))$, en utilisant une queue de priorité (monceau).

# Mise en pratique!

## En guise d'introduction

On vous demande de concevoir un algorithme d'apprentissage permettant d'identifier des fleurs sur un convoyeur. Il s'agit de trois variétés d'iris. Le convoyeur est doté d'une caméra capable de mesurer les longueurs et largeurs des pétales et sépales de chaque fleur. C'est à partir de ces caractéristiques (traits) que vous devez déterminer la sorte de chaque fleur (la classe). Vous ne connaissez rien aux fleurs! Fort heureusement vous disposez d'un ensemble d'entraînement associant à divers exemples de mesures d'iris la bonne variété (classe). Armé de l'algo 1-PPV et de Python vous foncez tête baissée.

## Comment calculer une distance $L^p$ (Minkowski) entre deux vecteurs

Passons maintenant aux choses sérieuses. On désire obtenir la fonction `minkowski_vec` qui nous permet de comparer deux fleurs sur la base de leurs traits. Complétez la fonction suivante, puis testez-la sur deux exemples d'iris (revoir au besoin la démo 2 pour l'accès et l'importation de l'ensemble de données iris). 



In [1]:
import numpy as np

In [2]:
def minkowski_vec(x1,x2,p=2.0):
    dist = np.sum(np.abs(x1 - x2)**p)**(1. / p)
    return dist

# pour tester
a = np.ones((10,5))
b = np.zeros((10,5))
print(minkowski_vec(a[0],b[0]))
print(minkowski_vec(a[0],a[0]))

2.23606797749979
0.0


Rappelez-vous la [définition](http://en.wikipedia.org/wiki/Minkowski_distance) de la distance. 

**Chose importante:** on peut calculer cette distance en itérant sur chaque composante de x1 et x2, et en calculant la somme après, ou on peut profiter du fait que la plupart (ou même toutes) des opérations mathématiques (abs, +, -, \*\* etc.) sur des structures de données itérables (listes, vecteurs/matrices) produisent des résultats équivalents à l'application d'une boucle for, mais en beaucoup moins de temps (on parle des langages interprétés comme Python). Par exemple, on peut calculer la somme de la différence des valeurs absolues de x1 et x2 comme

    s = 0
    for i in range(x1.shape[0]):
        s = s + abs(x1[i] - x2[i])

ou simplement

    s = numpy.sum(numpy.abs(x1 - x2))

En plus d'être plus compacte, la deuxième option est beaucoup plus rapide (parce qu'elle fait appel à une implémentation efficace de sum et abs en C++).


## Comment calculer une distance $L^p$ entre un vecteur et une matrice

Il nous faut aussi une fonction qui va nous permettre de comparer une fleur avec tout un ensemble de fleurs, sur la base de leurs traits. On va maintenant modifier la fonction `minkowski` pour calculer une *distance* $L^p$ entre un vecteur et une matrice (c.a.d. une fonction qui va nous retourner un vecteur de distances $L^p$)

In [3]:
def minkowski_mat(x,Y,p=2.0):
    dist = np.sum(np.abs(x-Y)**p, axis = 1)**(1. / p)
    return dist

# pour tester
a = np.ones((10,5))
b = np.zeros((10,5))
print(minkowski_mat(a,b))
print(minkowski_mat(a,a))

[2.23606798 2.23606798 2.23606798 2.23606798 2.23606798 2.23606798
 2.23606798 2.23606798 2.23606798 2.23606798]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Comme pour `minkowski_vec`, il y a deux manières de procéder:
 
1. *Simple et inefficace:* en écrivant une boucle qui appelle la fonction `minkowski_vec(x,Y[i,:],p)` et garde les résultats dans le vecteur `dist`.
2. *Plus compliqué mais plus efficace:* en utilisant le fait que `numpy` fait quelque chose d'intelligent quand il évalue `x-Y`, car il retourne une matrice qui contient le vecteur `x-Y[i,:]` sur la rangée $i$ (le mécanisme qui rend cela possible s'appelle *broadcasting*). Voici une solution:
        def minkowski_mat(x,Y,p=2.0):
            diff = x - Y # diff sera une matrice
            absdiff = abs(diff) # absdiff sera une matrice
            powdiff = absdiff**p # powdiff sera une matrice
            s = numpy.sum(powdiff,axis=1) # calcule la somme de chaque rangée, s est un vecteur
            dist = s**(1.0/p) # dist sera un vecteur aussi
            return dist

    ou bien

        def minkowski_mat(x,Y,p=2.0):
            return (numpy.sum((abs(x-Y))**p,axis=1))**(1.0/p) 

Encore une fois, chose importante à retenir:  
**la  grande majorité des opérations vecteur-vecteur, vecteur-matrice ou bien matrice-matrice seront beaucoup plus efficaces en utilisant les
opérateurs numpy au lieu d'une boucle for.** (les raisons étant,  entre autres, le fait que python est un langage interprété et que numpy a  des implémentations très efficace pour certaines opérations vectorielles).
 
Vous avez peut-être remarqué que la différence entre les implémentations efficaces de `minkowski_vec` et `minkowski_mat` est seulement la partie: `axis=1`. L'exercice est de comprendre pourquoi il est nécessaire de spécifier sur quel *axe* on va faire la somme.

## 1-PPV

Enfin, nous y sommes! Complétez la fonction suivante qui predit l'espece de l'iris decrite par les caracteristiques x et vérifiez son efficacité. 

In [17]:
def ppv(x, data, target, p=2):
    dist = minkowski_mat(x, data, p)
    # trouver l'indice de la distance minimum
    # retourner la classe (dernière colonne iris) correspondant à l'indice trouvé
    flower_type = target[np.argmin(dist)]
    return flower_type

À noter ici que `x` est le vecteur de caractéristiques (sans étiquette) de l'exemple de test. Ayant en mains la fonction `minkowski_mat`, les choses devrait être simple, car dist va contenir un vecteur/liste des distances. En utilisant `numpy.argmin` on va trouver l'exemple/la fleur qui est la plus *proche* (dans le sens de minkowski) de `x` et ainsi, on va conclure que l'étiquette (prédite) de `x` est celle de cet exemple.

## Conclusion

Nous avons maintenant en notre possession toutes les composantes de l'algorithme 1-PPV. Il reste simplement à en faire l'assemblage et à tester le tout. 

>Rappelez vous que les fonctions définies dans les cellules de code précédentes sont accessibles dans toutes les cellules subséquentes une fois les précédentes exécutés.

Afin de tester votre implémentation, écrivez une boucle `for` qui appelle, pour chaque exemple `i`, la fonction `ppv(iris[i,:-1],iris,p)` et qui compare la classe prédite avec `iris[i,-1]` (la vrai étiquette). Les deux devraient toujours être les mêmes (pourquoi?).

In [19]:
from sklearn import datasets
iris = datasets.load_iris()

for i in range(len(iris.data)):
    if ppv(iris.data[i], iris.data,iris.target) == iris.target[i]:
        print('Yes')
    else:
        print('No')


Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes


## Bonus / trucs auxquels réfléchir pour la prochaine fois

* Divisez l'ensemble iris en deux - un ensemble d'entraînement qui contient 100 exemples (un sous-ensemble aléatoire!) et un ensemble de test qui contient le reste.
  * Utilisez le premier sous-ensemble comme données sur lesquelles on va calculer les distances minkowski (donc données d'entraînement).
  * Calculez la performance de votre algorithme sur les deux ensembles. Pourquoi y a-t-il une telle différence?               



In [47]:
training_indices = np.random.choice(range(len(iris.data)), (100), replace = False)
test_indices = []
for i in range(len(iris.data)):
    if i not in training_indices:
        test_indices.append(i)
test_indices = np.array(test_indices)

training_data = np.array(iris.data[training_indices])
training_target = np.array(iris.target[training_indices])
test_data = np.array(iris.data[test_indices])
test_target_verif = np.array(iris.target[test_indices])

print('Comparaison du dataset avec lui-meme:')
for i in range(len(training_data)):
    if ppv(training_data[i], training_data, training_target) == training_target[i]:
        print('Yes')
    else:
        print('No')
print('\nComparaison des points tests avec le dataset:')
for i in range(len(test_data)):
    if ppv(test_data[i], training_data, training_target) == test_target_verif[i]:
        print('Yes')
    else:
        print('No')

Comparaison du dataset avec lui-meme:
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes

Comparaison des points tests avec le dataset:
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
Yes
Yes
No
Yes
Yes


* Implémentez l'algorithme $k$-PPV avec $k > 1$
   * Trouvez le $k$ qui donne la meilleure performance sur les deux ensembles - expliquez la différence.

* Que se passe-t-il lorsque $k=100$?